# Day 7

## part 1

- Elephants stole the operators! (lol, wat?!)
- `+` and `*` can  e inserted
- operations are evaluated left-to-right
- find the sum of the test values from the equations that are possible to solve.

In [34]:
from dataclasses import dataclass
import logging

from tqdm import  tqdm

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig as PC, markdown
)

log = logging.getLogger('day 7')
logging.basicConfig(level=logging.INFO)

In [23]:
parser = PC('\n', PC(': ', [int, PC(' ', int)]))

example = parse_from_file('day_7_example.txt', parser)

@dataclass
class Equation:
    total: int
    operands: list[int]
    operators: list[str] = None

    @property
    def combinations(self) -> list[list[str]]:
        """
        returns an iterable of all combinations of + and * operators for the
        equation
        """
        ops = ['+', '*']
        slots = len(self.operands) - 1
        log.debug(f'genearting combinations for {slots=}')
        combs = []
        for value in range(pow(2, slots)):
            temp = []
            for char in f'{value:0{slots}b}':
                temp.append(ops[int(char)])
            combs.append(temp)
        log.debug(f'{len(combs)} operator combinations generated')
        return combs
    
    def solve(self, operators: list[str]) -> int:
        """returns the result from applying the operands"""
        log.debug(f'solving with {operators=}')
        total = self.operands[0]
        log.debug(f'{total=}')
        ops = [op for op in operators]
        for value in self.operands[1:]:
            op = ops.pop(0)
            match(op):
                case '+':
                    total += value
                case '*':
                    total *= value
                case _:
                    ValueError(f'Operator not recognised: {op}')
            log.debug(f'{op}= {value} -> {total=}')
        return total
    
    def find_ops(self) -> None:
        """tests operands to try and solve the equation"""
        log.debug(f'attempting to solve: {self}')
        for combination in self.combinations:
            log.debug(f'testing {combination=}')
            if self.solve(combination) == self.total:
                self.operators = combination
                log.debug('solution found!')
                break
        else:
            log.debug('no solution found!')


example_eqs = [Equation(t, o) for t, o in example]

INFO:advent_of_code_utils.py:9 items loaded from "day_7_example.txt"


In [24]:

log.setLevel(logging.DEBUG)
for eq in example_eqs[1:3]:
    eq.find_ops()
    log.info(f'updated equation: {eq}')

DEBUG:day 7:attempting to solve: Equation(total=3267, operands=[81, 40, 27], operators=None)
DEBUG:day 7:genearting combinations for slots=2
DEBUG:day 7:4 operator combinations generated
DEBUG:day 7:testing combination=['+', '+']
DEBUG:day 7:solving with operators=['+', '+']
DEBUG:day 7:total=81
DEBUG:day 7:+= 40 -> total=121
DEBUG:day 7:+= 27 -> total=148
DEBUG:day 7:testing combination=['+', '*']
DEBUG:day 7:solving with operators=['+', '*']
DEBUG:day 7:total=81
DEBUG:day 7:+= 40 -> total=121
DEBUG:day 7:*= 27 -> total=3267
DEBUG:day 7:solution found!
INFO:day 7:updated equation: Equation(total=3267, operands=[81, 40, 27], operators=['+', '*'])
DEBUG:day 7:attempting to solve: Equation(total=83, operands=[17, 5], operators=None)
DEBUG:day 7:genearting combinations for slots=1
DEBUG:day 7:2 operator combinations generated
DEBUG:day 7:testing combination=['+']
DEBUG:day 7:solving with operators=['+']
DEBUG:day 7:total=17
DEBUG:day 7:+= 5 -> total=22
DEBUG:day 7:testing combination=['*'

In [31]:
# cool so let's try solving the exmaple
log.setLevel(logging.INFO)
for eq in example_eqs:
    eq.find_ops()
example_total_cal = sum(
    (eq.total for eq in example_eqs if eq.operators is not None))
log.info(f'the total example calibration is: {example_total_cal}')

INFO:day 7:the total example calibration is: 3749


In [35]:
# ok nice that works so let's solve for real!
puzzle_input = parse_from_file('day_7.txt', parser)
equations = [Equation(t, o) for t, o in puzzle_input]
log.info(f'loaded {len(equations)} equations')
for eq in tqdm(equations, desc='finding operators'):
    eq.find_ops()
log.info('finding total calibration value')
total_cal = sum(
    (eq.total for eq in equations if eq.operators is not None))
markdown(f'the total calibration result is: {total_cal}')

INFO:advent_of_code_utils.py:850 items loaded from "day_7.txt"
INFO:day 7:loaded 850 equations
finding operators: 100%|██████████| 850/850 [00:04<00:00, 180.26it/s]
INFO:day 7:finding total calibration value


the total calibration result is: 5702958180383

## part 2

- `||` yo new op just dropped! It concatenates numbers.
- Recompute theh total calibration result with this new operator in the mix

In [36]:
# the ones that already solve don't need to be reconsidered
ex_subset = [eq for eq in example_eqs if eq.operators is None]

In [46]:
class AdvancedEquation(Equation):
    """like an equation but more!"""
    def solve(self, operators: list[str]) -> int:
        """returns the result from applying the operands"""
        log.debug(f'solving with {operators=}')
        total = self.operands[0]
        log.debug(f'{total=}')
        ops = [op for op in operators]
        for value in self.operands[1:]:
            op = ops.pop(0)
            match(op):
                case '+':
                    total += value
                case '*':
                    total *= value
                case '||':
                    total = int(f'{total}{value}')
                case _:
                    ValueError(f'Operator not recognised: {op}')
            log.debug(f'{op}= {value} -> {total=}')
        return total
    
    @property
    def combinations(self) -> list[list[str]]:
        """
        returns an iterable of all combinations of +, * and || operators for
        the equation
        """
        ops = ['+', '*', '||']
        slots = len(self.operands) - 1
        log.debug(f'genearting combinations for {slots=}')
        combs = []
        # this should work up to base 10 - not that it needs to!
        for value in range(pow(len(ops), slots)):
            # for each value generate a string in that base
            temp = value
            digit_str = ''
            for power in reversed(range(slots)):
                count = 0
                sub = pow(len(ops), power)
                while temp - sub >= 0:
                    temp -= sub
                    count += 1
                digit_str += str(count)
            log.debug(f'{digit_str=}')
            # then we can do the same as before assigning an op per each digit
            temp = []
            for char in digit_str:
                temp.append(ops[int(char)])
            combs.append(temp)
        log.debug(f'{len(combs)} operator combinations generated')
        return combs

log.setLevel(logging.DEBUG)
ex_ad_eqs = [AdvancedEquation(eq.total, eq.operands) for eq in ex_subset]
for eq in ex_ad_eqs:
    eq.find_ops()
new_ex_total = sum((
    eq.total for eq in ex_ad_eqs if eq.operators is not None
))
log.info(
    f'{new_ex_total=} so combined total is {new_ex_total + example_total_cal}')

DEBUG:day 7:attempting to solve: AdvancedEquation(total=83, operands=[17, 5], operators=None)
DEBUG:day 7:genearting combinations for slots=1
DEBUG:day 7:digit_str='0'
DEBUG:day 7:digit_str='1'
DEBUG:day 7:digit_str='2'
DEBUG:day 7:3 operator combinations generated
DEBUG:day 7:testing combination=['+']
DEBUG:day 7:solving with operators=['+']
DEBUG:day 7:total=17
DEBUG:day 7:+= 5 -> total=22
DEBUG:day 7:testing combination=['*']
DEBUG:day 7:solving with operators=['*']
DEBUG:day 7:total=17
DEBUG:day 7:*= 5 -> total=85
DEBUG:day 7:testing combination=['||']
DEBUG:day 7:solving with operators=['||']
DEBUG:day 7:total=17
DEBUG:day 7:||= 5 -> total=175
DEBUG:day 7:no solution found!
DEBUG:day 7:attempting to solve: AdvancedEquation(total=156, operands=[15, 6], operators=None)
DEBUG:day 7:genearting combinations for slots=1
DEBUG:day 7:digit_str='0'
DEBUG:day 7:digit_str='1'
DEBUG:day 7:digit_str='2'
DEBUG:day 7:3 operator combinations generated
DEBUG:day 7:testing combination=['+']
DEBUG:d

In [52]:
# great that works! let's solve!
log.setLevel(logging.INFO)
ad_equations = [
    AdvancedEquation(eq.total, eq.operands) for eq in equations
    if eq.operators is None
]
log.info(f'converted {len(ad_equations)} to AdvancedEquation')
for eq in tqdm(ad_equations, desc='finding operators'):
    # had to run this again so just skip ones that are solved
    if eq.operators is not None:
        continue
    eq.find_ops()


INFO:day 7:converted 518 to AdvancedEquation
finding operators: 100%|██████████| 518/518 [04:03<00:00,  2.12it/s]


In [53]:
log.info('finding additional total calibration value')
ad_total_cal = sum(
    (eq.total for eq in ad_equations if eq.operators is not None))
log.info(f'The advanced total cal result is: {ad_total_cal}')
markdown(f'the new total calibration result is: {total_cal + ad_total_cal}')

INFO:day 7:finding additional total calibration value
INFO:day 7:The advanced total cal result is: 86909427938755


the new total calibration result is: 92612386119138